# Import Libraries

In [1]:
import os
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from src.data_loader.freihand_loader2 import F_DB2
from src.data_loader.sample_augmenter import SampleAugmenter
from src.constants import MASTER_THESIS_DIR, FREIHAND_DATA
from src.utils import read_json
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import torchvision
from easydict import EasyDict as edict
from src.utils import read_json
import torch
from torch import nn
from src.models.simclr_model import SimCLR

# Reading data

In [2]:
augmenter1 = SampleAugmenter(crop=True,resize=True, rotate=True, seed =1)
augmenter2 = SampleAugmenter(crop=True,resize=True, rotate=True,seed =7)
# train_param = edict(read_json(f"{MASTER_THESIS_DIR}/src/experiments/training_config.json"))
data = F_DB2(root_dir=os.path.join(FREIHAND_DATA, "training", "rgb"),
    labels_path=os.path.join(FREIHAND_DATA, "training_xyz.json"),
    camera_param_path=os.path.join(FREIHAND_DATA, "training_K.json"),
    transform=torchvision.transforms.ToTensor(),
    augmenter1=augmenter1,
    augmenter2=augmenter2)

# Visualization

In [3]:
@interact(id=widgets.IntSlider(min=1, max=100, step=5, value=3))
def visualize(id):
    sample = data[id]
#     joints = sample["joints"]
    img1 = sample["transformed_image1"]
    img2 = sample["transformed_image2"]
#     fig = plt.figure(figsize=(10, 10))
#     ax = fig.add_subplot(111)
    plt.imshow(torchvision.transforms.ToPILImage()(img1))
    plt.show()
    plt.imshow(torchvision.transforms.ToPILImage()(img2))


interactive(children=(IntSlider(value=3, description='id', min=1, step=5), Output()), _dom_classes=('widget-in…

# SimCLR model definition

and training

In [4]:
model = SimCLR()
data_loader = DataLoader(data,batch_size=128, num_workers=8)
val_data_loader = DataLoader(data, batch_size=128, num_workers=4)

In [ ]:
trainer = pl.Trainer(gpus=-1)

In [ ]:
trainer.fit(model, data_loader,val_data_loader)
torch.cuda.empty_cache()